In [ ]:
from app.MarketScraper import MarketScraper
from app.MariaDBClient import MariaDBClient
import pandas as pd
from app.tasks import fetch_market_data

In [ ]:
mariadbclient = MariaDBClient()

In [ ]:
supermercados = mariadbclient.get_active_tables()
print(supermercados)

In [ ]:
resultados = [fetch_market_data.delay(super) for super in supermercados]


total_items = []
total_stats = []

for resultado in resultados:
    items_dicts, stats_dicts = resultado.get()
    total_items.append(pd.DataFrame(items_dicts))
    total_stats.append(pd.DataFrame(stats_dicts))


total_items_df = pd.concat(total_items, ignore_index=True)
total_stats_df = pd.concat(total_stats, ignore_index=True)

In [ ]:
# market_scraper = MarketScraper()
# total_items = []
# total_stats = []
# #TODO Esto hacer con Celery + Task asi no es secuencial y se tarda menos
# for supermercado in supermercados:
#     data = market_scraper.fetch_data(supermercado)
#     items = market_scraper.get_items(data)
#     total_items.append(items)
#     stas = market_scraper.get_stats(data)
#     total_stats.append(stas)

# total_items_df = pd.concat(total_items, ignore_index=True)
# total_stats_df = pd.concat(total_stats, ignore_index=True)

# #4m 46.5s

In [ ]:
display(total_items_df)

In [ ]:
total_items_df[total_items_df.duplicated(subset='id_producto', keep=False)].sort_values(by='id_producto')



In [ ]:
display(total_stats_df)

In [ ]:
mariadbclient.insert_into_productos(total_items_df)
mariadbclient.insert_into_db("webscraping_info", total_stats_df)
mariadbclient.insert_into_db("historico", total_items_df)